In [1]:
# ~~~~~~~~~~~~~~ Libraries
import sys, os
import mne # Python package for processing and analyzing electrophysiological data
import numpy as np
from glob import glob # look for all the pathnames matching a specified pattern according to the rules
import matplotlib.pyplot as plt
from mne.preprocessing import ICA # ICA (Independent Component Analysis) algorithm, which is for artifact removal
from autoreject import AutoReject # Python package for automatically rejecting bad epochs in EEG/MEG data
import json
import owncloud
import pandas as pd
import braindecode
import torch
import re
import pickle
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, cross_validate, StratifiedKFold

# Deep learning
from braindecode.models import EEGNetv4
from braindecode.preprocessing import exponential_moving_standardize
from braindecode.util import set_random_seeds
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split
from braindecode import EEGClassifier
# ~~~~~~~~~~~~~~ Libraries ~~~~~~~~~~~~~~

In [16]:
# ~~~~~~~~~~~~~~ Parameters
group = 'adult'
modality = 'visual' # 'visual' or 'audio'
subject = 'sub-05'
condition = 'totaldot' # 'singledot', 'totaldot', 'circum'

# EEGNET parameters
# Load the JSON file
code_directory = os.path.dirname("/u/kazma/MINT/code/1st-level")
sys.path.append(code_directory) 
os.chdir(os.path.dirname("/u/kazma/MINT/code/1st-level"))
print(os.getcwd())
with open("config.json") as f: # import variables from config.json
    config = json.load(f) 
globals().update(config)


# Print out each parameter
print(f"{modality} data of {subject} is processed")
print(f"condition pair: {condition}")
print("EEGNET parameters:")
print(f"====== CV: {EEGNET_CV}")
print(f"====== Max epochs: {EEGNET_MAX_EPOCHS}")
print(f"====== Batch size: {EEGNET_BATCH_SIZE}")

/viper/u/kazma/MINT/code
visual data of sub-05 is processed
condition pair: totaldot
EEGNET parameters:
====== CV: 10
====== Max epochs: 200
====== Batch size: 16


In [17]:
# ~~~~~~~~~~~~~~ Set the working directory
path = f"{COMPUTE_DIR}/data/{group}/interim/{modality}"
sub_folders = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))]
sub_folders_sorted = sorted(sub_folders, key=lambda x: int(re.search(r'\d+', x).group())) # Sort the folders based on the numeric part after "sub-"
# ~~~~~~~~~~~~~~ Set the working directory ~~~~~~~~~~~~~~

# ~~~~~~~~~~~~~~ Concatanating 3 sessions
each_sub_path = f"{COMPUTE_DIR}/data/{group}/interim/{modality}/{subject}"
each_sub_folders = [f for f in os.listdir(each_sub_path) if os.path.isdir(os.path.join(each_sub_path, f))]
each_sub_folders_sorted = sorted(each_sub_folders, key=lambda x: int(re.search(r'\d+', x).group()))

eegdata_dict = {}
for x,ses_loop in enumerate(each_sub_folders_sorted):
    sub_filename = os.path.join(path, subject, ses_loop, 'epochs-epo.fif') 
    epochs = mne.read_epochs(sub_filename, preload=True)
    eegdata_dict[ses_loop] = epochs

# concatanate all 3 sessions into 1 epoch
epochs = mne.concatenate_epochs([eegdata_dict['ses-01'], eegdata_dict['ses-02'], eegdata_dict['ses-03']])
# ~~~~~~~~~~~~~~ Concatanating 3 sessions ~~~~~~~~~~~~~~



Reading /ptmp/kazma/DATA-MINT/data/adult/interim/visual/sub-05/ses-01/epochs-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1260 matching events found
No baseline correction applied
0 projection items activated
Reading /ptmp/kazma/DATA-MINT/data/adult/interim/visual/sub-05/ses-02/epochs-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1260 matching events found
No baseline correction applied
0 projection items activated
Reading /ptmp/kazma/DATA-MINT/data/adult/interim/visual/sub-05/ses-03/epochs-epo.fif ...
    Found the data of interest:
        t =    -100.00 ...    1000.00 ms
        0 CTF compensation matrices available
Not setting metadata
1260 matching events found
No baseline correction applied
0 projection items activated
Not setting metadata
3780 matching events found
Appl

/tmp/ipykernel_2029821/305156617.py:19: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  epochs = mne.concatenate_epochs([eegdata_dict['ses-01'], eegdata_dict['ses-02'], eegdata_dict['ses-03']])


In [4]:
# ~~~~~~~~~~~~~~ Concatanating 3 sessions csv file
each_sub_path = f"{COMPUTE_DIR}/data/{group}/raw/{modality}/{subject}"
each_sub_folders = [f for f in os.listdir(each_sub_path) if os.path.isdir(os.path.join(each_sub_path, f))]
each_sub_folders_sorted = sorted(each_sub_folders, key=lambda x: int(re.search(r'\d+', x).group()))

csv_dict = {}
for x,ses_loop in enumerate(each_sub_folders_sorted):
    csv_file = f"{subject}_{modality}_{ses_loop}.csv"
    sub_filename = os.path.join(each_sub_path, ses_loop, csv_file) 
    csv_df = pd.read_csv(sub_filename)
    csv_dict[ses_loop] = csv_df

# concatanate all 3 sessions into 1 epoch
csv_dfs = pd.concat([csv_dict['ses-01'], csv_dict['ses-02'], csv_dict['ses-03']], axis=0).reset_index(drop=True)
# ~~~~~~~~~~~~~~ Concatanating 3 sessions csv file~~~~~~~~~~~~~~



# ~~~~~~~~~~~~~~ Obtain target conditions' trials
csv_dfs['Extracted'] = csv_dfs['condition'].str.split('_').str[-1]
csv_idx = csv_dfs[['condition', 'Extracted']]
condition_lists = csv_idx['Extracted'].unique()

idx_to_keep = csv_dfs.index[csv_dfs['Extracted'] == condition].tolist()
# idx_to_keep = csv_dfs.index[csv_dfs['Extracted'].isin(condition)].tolist()

selected_epochs = epochs[idx_to_keep]
print(f"Selected trials: {len(selected_epochs)}")
print(f"Target condition: {condition}")
# ~~~~~~~~~~~~~~ Obtain target conditions' trials ~~~~~~~~~~~~~~

Selected trials: 1260
Target condition: totaldot


In [5]:
# Crop epochs to the desired time range
cropped_epochs = selected_epochs.copy().crop(tmin=EEGNET_MIN_TIME, tmax=EEGNET_MAX_TIME)

# Get the info about the cropped data
conditions = list(cropped_epochs.event_id.keys()) # list of conditions
n_conditions = len(conditions) # number of conditions
n_trials = len(cropped_epochs) # number of trials
n_samples = cropped_epochs.get_data().shape[2]
n_channels = cropped_epochs.get_data().shape[1]
min_time = cropped_epochs.times[0]*1000   # First time point in milli seconds
max_time = cropped_epochs.times[-1]*1000    # Last time point in milli seconds


print("=====================================================")
print("=====================================================")
print("=====================================================")

print(f" Condition lists: {conditions}")
print(f" Total trials: {n_trials}")
print(f" Time points: {n_samples}")
print(f" Time window (ms): {min_time} - {max_time}")


# check GPU availability
cuda = torch.cuda.is_available()  # Check if a GPU is available
device = "cuda" if cuda else "cpu"  # Use "cuda" if available, otherwise fallback to "cpu"

if cuda:
    torch.backends.cudnn.benchmark = True  # Enable cuDNN auto-tuning for performance


# Set random seeds
# PURPOSE: reproducibility across runs, random initializations (e.g., model weights) yield consistant results
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)


# dictionaries for decoding accuracy
pairwise_decoding_accuracies = {}
# dictionaries for decoding accuracy of standard deviation
pairwise_decoding_accuracies_std = {}
# dictionaries for estimators
pairwise_decoding_estimators = {}


 Condition lists: ['numerosity 1', 'numerosity 2', 'numerosity 3', 'numerosity 4', 'numerosity 5', 'numerosity 6']
 Total trials: 1260
 Time points: 251
 Time window (ms): 0.0 - 500.0


In [ ]:
for i in range(n_conditions):
    for j in range(i + 1, n_conditions):

            cond1 = conditions[i]
            cond2 = conditions[j]

            print(f"{cond1} vs. {cond2}")
            
            filtered_epochs = cropped_epochs[cond1,cond2]
            a1 = {cond: (filtered_epochs.events[:, 2] == event_id).sum() 
                for cond, event_id in filtered_epochs.event_id.items()}

            # Print the number of trials per condition
            print(f"Trials per condition:\n{a1}")

            # get EEG data
            X = filtered_epochs.get_data(picks=mne.pick_types(filtered_epochs.info, eeg=True, eog=False, exclude='bads'))

            # get labels (=numerosity)
            unique_labels = np.unique(filtered_epochs.events[:,-1])
            label_0, label_1 = unique_labels[0], unique_labels[1]  # Assign the first label to 0 and the second to 1

            y = np.where(filtered_epochs.events[:, -1] == label_0, 0, 1)

            skfold = StratifiedKFold(n_splits=EEGNET_CV, shuffle=True, random_state=23)

            # exp. moving std. for each trial
            for s in range(X.shape[0]):
                X[s,:,:] = exponential_moving_standardize(X[s,:,:], factor_new=0.001, init_block_size=None, eps=1e-4)

            # create the model
            net = EEGClassifier(
                "EEGNetv4", 
                            module__n_chans=n_channels, # Number of EEG channels
                            module__n_outputs=2,               # Number of outputs of the model. This is the number of classes in the case of classification.
                            module__n_times=n_samples,         # Number of time samples of the input window.
                            module__final_conv_length='auto',  # Length of the final convolution layer. If "auto", it is set based on the n_times.
                            module__pool_mode='mean',          # Pooling method to use in pooling layers
                            module__F1=8,                      # Number of temporal filters in the first convolutional layer.
                            module__D=2,                       # Depth multiplier for the depthwise convolution.
                            module__F2=16,                     # Number of pointwise filters in the separable convolution. Usually set to ``F1 * D``.
                            module__kernel_length=64,         # Length of the temporal convolution kernel. Usally sampling rate / 2 = 500/2 = 250
                            module__third_kernel_size=(8, 4), 
                            module__drop_prob=0.25,            # Dropout probability after the second conv block and before the last layer. 0.5 for within-subject classification, 0.25 in cross-subject clasification
                            module__chs_info=None,             # (list of dict) – Information about each individual EEG channel. This should be filled with info["chs"]. Refer to mne.Info for more details.
                            module__input_window_seconds=None, # Length of the input window in seconds.
                            module__sfreq=SFREQ,
                            max_epochs=EEGNET_MAX_EPOCHS,
                            batch_size=EEGNET_BATCH_SIZE,
                            train_split=None,
            )


            cvs = cross_validate(net, 
                                X, 
                                y, 
                                scoring="accuracy", # for balanced classes, this corresponds to accuracy,
                                # chance level might be 0 (adjusted = False), or 0.X (adjusted = True)
                                cv=skfold, 
                                n_jobs=-1, # only 1 to avoid overload in parallel jobs, in non-par jobs it could be -1
                                return_estimator=True, # if you need the model to estimate on another test set
                                return_train_score=False,
                                )
            
            pairwise_decoding_accuracies[(cond1,cond2)] = np.mean(cvs['test_score'])
            pairwise_decoding_accuracies_std[(cond1,cond2)] = np.std(cvs['test_score'])
            pairwise_decoding_estimators[(cond1,cond2)] = cvs['estimator']


# ~~~~~~~~~~~~~~~~ Save the decoding accuracy
save_folder = f"{COMPUTE_DIR}/data/{group}/processed/{modality}/{subject}"

# concatenated_str = "_".join(condition)
filename = f"EEGNet_accuracy_{condition}.pkl"
save_path = os.path.join(save_folder, filename) #  a pickle file
with open(save_path, "wb") as f:
    pickle.dump(pairwise_decoding_accuracies, f)
print(f"{subject}: saved in {filename}")
# ~~~~~~~~~~~~~~~~ Save the decoding accuracy ~~~~~~~~~~~~~~~~

# ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation
filename = f"EEGNet_std_{condition}.pkl"
save_path = os.path.join(save_folder, filename) #  a pickle file
with open(save_path, "wb") as f:
    pickle.dump(pairwise_decoding_accuracies_std, f)
print(f"{subject}: saved in {filename}")
# ~~~~~~~~~~~~~~~~ Save the decoding accuracy standard deviation ~~~~~~~~~~~~~~~~


# ~~~~~~~~~~~~~~~~ Save the EEGNet estimation
filename = f"EEGNet_estimator_{condition}.pkl"
save_path = os.path.join(save_folder, filename) #  a pickle file
with open(save_path, "wb") as f:
    pickle.dump(pairwise_decoding_estimators, f)
print(f"{subject}: saved in {filename}")
# ~~~~~~~~~~~~~~~~ Save the EEGNet estimation ~~~~~~~~~~~~~~~~

numerosity 1 vs. numerosity 2
Trials per condition:
{'numerosity 1': 213, 'numerosity 2': 213}


/viper/ptmp/kazma/miniforge3/envs/MINT/lib/python3.9/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs

  epoch    train_loss     dur
-------  ------------  ------
      1        0.7446  1.8924
      2        0.6940  1.8126
      3        0.6865  1.8027
      4        0.6326  1.8872
      5        0.6262  2.3999
      6        0.6265  2.1083
      7        0.6378  2.2816
      8        0.6302  2.2017
      9        0.5829  2.2011
     10        0.5910  2.5109
     11        0.5981  2.5915
     12        0.5863  2.5064


/viper/ptmp/kazma/miniforge3/envs/MINT/lib/python3.9/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs

     13        0.5888  2.5892


/viper/ptmp/kazma/miniforge3/envs/MINT/lib/python3.9/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs

  epoch    train_loss     dur
-------  ------------  ------
      1        0.8202  2.5893
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7829  2.5986
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7731  2.6169
  epoch    train_loss     dur
-------  ------------  ------
      1        0.8134  2.6115
     14        0.5801  2.6109
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7357  2.7992
  epoch    train_loss     dur
-------  ------------  ------
      1        0.8059  2.6969
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7905  2.6951
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7595  2.7866
  epoch    train_loss     dur
-------  ------------  ------
      1        0.7601  2.9049
      2        0.7179  2.4988
      2        0.7011  2.5077
      2        0.7035  2.6061
      2        0.7300  2.6026
     15        0.5696  2.6009
      2   